# Limpieza de dataset del Censo Nacional de Gobiernos Municipales y Delegacionales 2015
## 1. Introduccion

Indicadores que salen de este dataset:

## 2. Descarga de datos

In [13]:
descripciones = {
'P0306' : 'Programas de modernización catastral',
'P0307' : 'Disposiciones normativas sustantivas en materia de desarrollo urbano u ordenamiento territorial',
'P1001' : 'Promedio diario de RSU recolectados',
'P1003' : 'Número de municipios con disponibilidad de servicios relacionados con los RSU',
'P1006' : 'Número de municipios con aplicación de programas locales orientados a la GIRSU',
'P1009' : 'Número de municipios con estudios de generación de RSU',
}



In [36]:
# Librerias utilizadas
import pandas as pd
import sys
import urllib
import os
import zipfile
import csv
import pprint

In [2]:
# Configuracion del sistema
print('Python {} on {}'.format(sys.version, sys.platform))
print('Pandas version: {}'.format(pd.__version__))
import platform; print('Running on {} {}'.format(platform.system(), platform.release()))

Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)] on win32
Pandas version: 0.20.1
Running on Windows 8.1


In [3]:
root = r'http://www.beta.inegi.org.mx/contenidos/proyectos/censosgobierno/municipal/cngmd/2015/datosabiertos/'

links = {
    'P0306' : r'm1/Programa_modernizacion_catastral_cngmd2015_csv.zip', # Programas de modernización catastral
    'P0307' : r'm2/Marco_regulatorio_cngmd2015_csv.zip', # Disposiciones normativas sustantivas en materia de desarrollo urbano u ordenamiento territorial
    'P1001' : r'm6/Rec_RSU_cngmd2015_csv.zip', # Promedio diario de RSU recolectados
    'P1006' : r'm6/Prog_gest_int_RSU_cngmd2015_csv.zip', # Número de municipios con aplicación de programas locales orientados a la GIRSU
    'P1009' : r'm6/Est_gen_comp_RSU_cngmd2015_csv.zip', # Número de municipios con estudios de generación de RSU
}

En el caso del parámetro P1003, los datos se extraen desde 3 archivos. Estos archivos son una base de datos para cada servicio relacionado con los RSU, Utilizando nuevamente el archivo que utiliza P1001 y dos adicionales:

In [4]:
P1003links = { # Número de municipios con disponibilidad de servicios relacionados con los RSU
    1 : r'm6/Rec_RSU_cngmd2015_csv.zip',
    2 : r'm6/Trat_RSU_cngmd2015_csv.zip',
    3 : r'm6/Disp_final_RSU_cngmd2015_csv.zip'
}

In [5]:
# Destino local
destino = r'D:\PCCS\00_RawData\01_CSV\cngmd\2015'

In [6]:
# Descarga de zips para parametros de un solo archivo
m_archivos = {}   # Diccionario para guardar memoria de descarga
for parametro, fuente in links.items():
    file = fuente.split('/')[1]
    remote_path = root+fuente
    local_path = destino + r'\{}'.format(file)
    if os.path.isfile(local_path):
        print('Ya existe el archivo: {}'.format(local_path))
        m_archivos[parametro] = local_path
    else:
        print('Descargando {} ... ... ... ... ... '.format(local_path))
        urllib.request.urlretrieve(remote_path, local_path) #
        m_archivos[parametro] = local_path
        print('se descargó {}'.format(local_path))
    

Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Programa_modernizacion_catastral_cngmd2015_csv.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Marco_regulatorio_cngmd2015_csv.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Rec_RSU_cngmd2015_csv.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Prog_gest_int_RSU_cngmd2015_csv.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Est_gen_comp_RSU_cngmd2015_csv.zip


In [7]:
# Descarga de zips para parametro P1003
m_archivos2 = {}   # Diccionario para guardar memoria de descarga
for parametro, fuente in P1003links.items():
    file = fuente.split('/')[1]
    remote_path = root+fuente
    local_path = destino + r'\{}'.format(file)
    if os.path.isfile(local_path):
        print('Ya existe el archivo: {}'.format(local_path))
        m_archivos2[parametro] = local_path
    else:
        print('Descargando {} ... ... ... ... ... '.format(local_path))
        urllib.request.urlretrieve(remote_path, local_path) #
        m_archivos2[parametro] = local_path
        print('se descargó {}'.format(local_path))
    

Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Rec_RSU_cngmd2015_csv.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Trat_RSU_cngmd2015_csv.zip
Ya existe el archivo: D:\PCCS\00_RawData\01_CSV\cngmd\2015\Disp_final_RSU_cngmd2015_csv.zip


In [8]:
# Descompresión de archivos de m_parametro
unzipped = {}
for parametro, comprimido in m_archivos.items():
    target = destino + '\\' + parametro
    if os.path.isfile(target):
        print('Ya existe el archivo: {}'.format(target))
        unzipped[parametro] = target
    else:
        print('Descomprimiendo {} ... ... ... ... ... '.format(target))
        descomprimir = zipfile.ZipFile(comprimido, 'r')
        descomprimir.extractall(target)
        descomprimir.close
        unzipped[parametro] = target
        

Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P0306 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P0307 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1001 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1006 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1009 ... ... ... ... ... 


In [9]:
# Descompresión de archivos de m_parametro2
unzipped2 = {}
for parametro, comprimido in m_archivos2.items():
    target = destino + '\\P1003\\' + str(parametro)
    if os.path.isfile(target):
        print('Ya existe el archivo: {}'.format(target))
        unzipped2[parametro] = target
    else:
        print('Descomprimiendo {} ... ... ... ... ... '.format(target))
        descomprimir = zipfile.ZipFile(comprimido, 'r')
        descomprimir.extractall(target)
        descomprimir.close
        unzipped2[parametro] = target
        

Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1003\1 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1003\2 ... ... ... ... ... 
Descomprimiendo D:\PCCS\00_RawData\01_CSV\cngmd\2015\P1003\3 ... ... ... ... ... 


In [21]:
# Localizacion de archivos de cada parametro
# Cada parametro tiene rutas y estructuras distintas. En este paso localizo manualmente 
# cada tabla y estructura desde los comprimidos. cada valor del diccionario contiene la ruta hacia
# donde se encuentran las tablas.

cd = r'\conjunto_de_datos'

tablas = {
    'P0306' : destino + r'\P0306' + cd,
    'P0307' : destino + r'\P0307\marco_regulatorio_cngmd2015_dbf' + cd,
    'P1001' : destino + r'\P0307\Rec_RSU_cngmd2015_csv' + cd,
    'P1006' : destino + r'\P1006\Prog_gest_int_RSU_cngmd2015_csv' + cd,
    'P1009' : destino + r'\P1006\Est_gen_comp_RSU_cngmd2015_csv' + cd,
}

# Tablas para P1003
destino2 = destino + r'\P0306'
tablasP1003 = {
    '1' : destino2 + r'\1' + r'\Rec_RSU_cngmd2015_csv' + cd,
    '2' : destino2 + r'\2' + r'\Trat_RSU_cngmd2015_csv' + cd,
    '3' : destino2 + r'\3' + r'\Disp_final_RSU_cngmd2015_csv' + cd,
}

## Construccion de datasets estándar
Los datasets para cada parámetro surgen de diferentes preguntas del censo por lo que sus estructuras son muy desemejantes, razon por la cual:
(1) : Cada parámetro tiene que procesarse individualmente.
(2) : Es conveniente extraer de manera individual los metadatos de cada parámetro. Con este propósito, el siguiente script sirve para extraer los metadatos de cada dataset:

In [135]:
# Script para extraer metadatos:
def getmeta(path):   # Path es el contenido en las variables 'tablas' para cada parametro
    cat = r'\catalogos'
    dic = r'\diccionario_de_datos'
    metadict = {}
    metapath = path.replace(cd, cat)
    metafiles = os.listdir(metapath)
    dicdict = {}
    dicpath = path.replace(cd, dic)
    dicfiles = os.listdir(dicpath)
    for file in metafiles:
        variable = file.replace('.csv', '')
        if file.endswith('.csv'):
            csvpath = metapath+'\\'+file
            metadf = pd.DataFrame.from_csv(csvpath, parse_dates=False)
            try:
                metadf.index = metadf.index.map(str.lower)
            except:
                pass
            metadict[variable] = metadf
        else:
            dothis = input('El archivo {} no es csv, que deseas hacer? [DD]etener [CC]ontinuar'.format(file))
            dothis = dothis.lower()
            if dothis == 'dd':
                raise GeneratorExit('Script detenido por el usuario')
            elif dothis == 'cc':
                continue
            else:
                raise KeyError('No entendi la instruccion {}'.format(dothis))
    for file in dicfiles:
        if file.endswith('.csv'):
            filename = file.replace('.csv', '')
            csvpath = dicpath+'\\'+file
            dicdf = pd.read_csv(csvpath, skiprows=2, usecols=[1, 2], index_col=0, parse_dates=False).dropna()
            dicdf.index = dicdf.index.map(str.lower)
            dicdict[filename] = dicdf
    
    return dicdict, metadict

In [216]:
# Funcion para revisar metadatos
def queryvar(var, tablelen=10, colprint = 125):
    pdefault = pd.get_option('display.max_colwidth')
    pd.set_option('display.max_colwidth', colprint)   # Expande el espacio para imprimir columnas
    print('"{}" :\n{}'.format(var, p0306dic.loc[var][0].upper()))
    if len(p0306meta[var]) > tablelen:
        print('{}\nImprimiendo {} de {} registros'.format('-'*40,tablelen, len(p0306meta[var])))
    print(p0306meta[var].head(tablelen))
    pd.set_option('display.max_colwidth', pdefault)   # Regresa la variable de impresion de columnas a su default

**P0306 Programas de modernización catastral**

Existencia de programas de modernización catastral en los municipios. 

In [371]:
# Creacion de diccionarios con metadatos para cada variable de P0306:
par = 'P0306'
p0306dic, p0306meta = getmeta(tablas['P0306'])
print('Se extrajeron metadatos para las siguientes variables de {}:'.format(par))
for key in p0306meta.keys(): print(key)
print('\nDiccionarios disponibles para {}:'.format(par))
for key in p0306dic.keys(): print(key)

Se extrajeron metadatos para las siguientes variables de P0306:
acc_modr
estructu
inst_enc
perio_ac
prog_mod
tip_inst
ubic_geo

Diccionarios disponibles para P0306:
diccionario_de_datos_programa_modernizacion_catastral_cngmd2015_dbf


In [372]:
# Para P0306, solo existe una tabla de descripciones por lo que se convierte a un dataframe unico para poder indexar
p0306dic = p0306dic['diccionario_de_datos_programa_modernizacion_catastral_cngmd2015_dbf']
p0306dic

,Descripción
Nombre de la columna,
ubic_geo,Clasificación de las entidades federativas y d...
prog_mod,Clasificación de los municipios de acuerdo con...
inst_enc,Clasificación de los municipios de acuerdo con...
tip_inst,Clasificación de los municipios de acuerdo con...
acc_modr,Clasificación de los municipios de acuerdo con...
perio_ac,Clasificación de los municipios de acuerdo con...
estructu,Clasificación de los municipios y las delegaci...


In [373]:
list(p0306dic.index)

['ubic_geo',
 'prog_mod',
 'inst_enc',
 'tip_inst',
 'acc_modr',
 'perio_ac',
 'estructu']

In [374]:
print('** Descripciones de variables **\n'.upper())
for i in p0306dic.index:
    queryvar(i)
    print('\n')

** DESCRIPCIONES DE VARIABLES **

"ubic_geo" :
CLASIFICACIÓN DE LAS ENTIDADES FEDERATIVAS Y DE ÉSTAS A SUS MUNICIPIOS.
----------------------------------------
Imprimiendo 10 de 2542 registros
          ent    desc_ent  mpio                       des_mpio
ubic_geo                                                      
11034      11  Guanajuato    34                 Santa Catarina
11035      11  Guanajuato    35  Santa Cruz de Juventino Rosas
11036      11  Guanajuato    36             Santiago Maravatío
11037      11  Guanajuato    37           Silao de la Victoria
11038      11  Guanajuato    38                    Tarandacuao
11039      11  Guanajuato    39                       Tarimoro
11040      11  Guanajuato    40                  Tierra Blanca
11041      11  Guanajuato    41                      Uriangato
11042      11  Guanajuato    42              Valle de Santiago
11043      11  Guanajuato    43                       Victoria


"prog_mod" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE A

In [375]:
# Carga de datos
P0306f = tablas['P0306']+'\\'+os.listdir(tablas['P0306'])[0]
df = pd.read_csv(P0306f, dtype={'ubic_geo':'str'}, index_col=0)
P0306 = df.where((pd.notnull(df)), None)

El archivo está estructurado de manera inconveniente, teniendo un renglón para cada variable. Lo conveniente es que cada renglón contenga toda la información de un solo municipio.

In [376]:
# subset para pruebas
test = P0306.loc[15045]
test

,prog_mod,inst_enc,tip_inst,acc_modr,perio_ac,estructu
ubic_geo,,,,,,
15045,1,None,None,None,None,240500
15045,None,1,1,None,None,240501
15045,None,1,2,None,None,240501
15045,None,2,2,None,None,240501
15045,None,None,None,None,2,240503
15045,None,None,None,1,None,240502
15045,None,None,None,2,None,240502
15045,None,None,None,4,None,240502
15045,None,None,None,5,None,240502


Para corregirlo, primero hacemos dataframes separados para cada variable. Afortunadamente, la columna 'Estructu' sirve para agrupar estructuralmente el dataframe

In [377]:
queryvar('estructu')

"estructu" :
CLASIFICACIÓN DE LOS MUNICIPIOS Y LAS DELEGACIONES DEL DISTRITO FEDERAL DE ACUERDO CON LA EXISTENCIA DEL PROGRAMA DE MODERNIZACIÓN CATASTRAL, ADEMÁS DE LA INSTITUCIÓN QUE COORDINA Y/O APOYA EL PROGRAMA DE MODERNIZACIÓN CATASTRAL, ASÍ COMO LAS ACCIONES DEL PROGRAMA DE MODERNIZACIÓN CATASTRAL REALIZADAS Y DEL PERIODO EN QUE SE REALIZARON.
                                                                                    descrip
estructu                                                                                   
240500                       El municipio cuenta con un programa de modernización catastral
240501            Institución que coordina y/o apoya el programa de modernización catastral
240502                          Acciones del programa de modernización catastral realizadas
240503    Periodo en que se realizaron las acciones del programa de modernización catastral


In [378]:
# ¿El municipio cuenta con un programa de modernización catastral?
P0306_00 = P0306[P0306['estructu'] == 240500]['prog_mod'].astype('int')
print(P0306_00.head(10))
print('-'*50)
queryvar('prog_mod')

ubic_geo
15045     1
15046     2
15047     2
15048     1
15049     2
15050     1
15051     1
15052     1
15053     1
15054    99
Name: prog_mod, dtype: int32
--------------------------------------------------
"prog_mod" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE ACUERDO CON EL PROGRAMA DE MODERNIZACIÓN CATASTRAL CON EL QUE CUENTA.
            desc_pro
prog_mod            
1                 Si
2                 No
99        No se sabe


In [379]:
# ¿En que periodo se realizaron las acciones del programa de modernización catastral?
P0306_03 = P0306[P0306['estructu'] == 240503]['perio_ac'].astype('int')
print(P0306_03.head(10))
print('-'*50)
queryvar('perio_ac')

ubic_geo
1006     1
2001     3
2002     4
2003     1
2004     1
2005     1
3002     2
3003    99
3008     1
3009     5
Name: perio_ac, dtype: int32
--------------------------------------------------
"perio_ac" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE ACUERDO CON EL PERIODO EN EL QUE SE REALIZARON LAS ACCIONES DEL PROGRAMA DE MODERNIZACIÓN CATASTRAL.
                                                    descrip
perio_ac                                                   
1                         En el transcurso del presente año
2                                       Durante el año 2014
3                                       Durante el año 2013
4                                        Antes del año 2013
5         No se han realizado acciones para atender el tema
99                                               No se sabe


In [383]:
# ¿Qué acciones se realizaron?
P0306_02 = P0306[P0306['estructu'] == 240502]['acc_modr'].astype('int').groupby('ubic_geo').apply(list)
print(P0306_02.head(10))
queryvar('acc_modr')

ubic_geo
1006                   [1, 5]
2001                [2, 3, 4]
2002       [1, 2, 3, 4, 5, 6]
2003             [1, 2, 3, 4]
2004    [1, 2, 3, 4, 5, 6, 7]
2005          [1, 2, 3, 4, 6]
3002       [1, 2, 3, 4, 5, 6]
3003                     [99]
3008       [1, 2, 3, 4, 5, 6]
3009                      [7]
Name: acc_modr, dtype: object
"acc_modr" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE ACUERDO CON LAS ACCIONES DEL PROGRAMA DE MODERNIZACIÓN CATASTRAL QUE SE HAN REALIZADO EN ELLOS.
                                                                                               desc_acc
acc_modr                                                                                               
1                                                                        Actualizar el padrón catastral
2                                                         Generar o actualizar la cartografía catastral
3                                                              Adquirir un sistema de gestión catastr

In [384]:
# ¿Cuantas acciones se realizaron?
P0306_02b = P0306_02.apply(len).rename('n_acc_modr')
P0306_02b.head(10)

ubic_geo
1006    2
2001    3
2002    6
2003    4
2004    7
2005    5
3002    6
3003    1
3008    6
3009    1
Name: n_acc_modr, dtype: int64

In [385]:
queryvar('inst_enc')

"inst_enc" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE ACUERDO CON LA INSTITUCIÓN ENCARGADA DE COORDINAR Y/O EJECUTAR EL PROGRAMA DE MODERNIZACIÓN CATASTRAL, ASÍ COMO DE LA QUE SE RECIBEN APOYOS.
                                                                                        desc_ins
inst_enc                                                                                        
1         Institución encargada de coordinar y/o ejecutar el programa de modernización catastral
2                                                        institución de la que se reciben apoyos


In [387]:
# ¿Que instituciones se han involucrado en la modernizacion catastral, y de qué manera?
P0306_01t = P0306[P0306['estructu'] == 240501][['inst_enc', 'tip_inst']]    # tipo de apoyo e institucion 
P0306_01t.head()

,inst_enc,tip_inst
ubic_geo,,
1006,1,4
1006,2,4
2001,1,3
2001,2,3
2002,1,3


Se reemplazarán numeros por descripciones en **tip_inst**:

In [388]:
queryvar('tip_inst')

"tip_inst" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE ACUERDO CON LAS ADMINISTRACIONES PÚBLICAS MUNICIPALES POR ENTIDAD FEDERATIVA SEGÚN INSTITUCIÓN QUE COORDINA EL PROGRAMA DE MODERNIZACIÓN CATASTRAL Y/O RECIBEN EL APOYO.
                                                                  descrip
tip_inst                                                                 
1                         Administración pública de la entidad federativa
2                 Banco Nacional de Obras y Servicios Públicos (BANOBRAS)
3         Secretaría de Desarrollo Agrario, Territorial y Urbano (SEDATU)
4                                                                    Otra


In [389]:
# Institucion involucrada
instit = {
    1:'Administración pública de la entidad federativa',
    2:'BANOBRAS',
    3:'SEDATU',
    4:'OTRA INSTITUCION'
}

In [390]:
P0306_01t['tip_inst'] = P0306_01t['tip_inst'].replace(instit)
P0306_01t.head()

,inst_enc,tip_inst
ubic_geo,,
1006,1,OTRA INSTITUCION
1006,2,OTRA INSTITUCION
2001,1,SEDATU
2001,2,SEDATU
2002,1,SEDATU


Y se separará la columna 'inst_enc' en 2:

In [391]:
queryvar('inst_enc')

"inst_enc" :
CLASIFICACIÓN DE LOS MUNICIPIOS DE ACUERDO CON LA INSTITUCIÓN ENCARGADA DE COORDINAR Y/O EJECUTAR EL PROGRAMA DE MODERNIZACIÓN CATASTRAL, ASÍ COMO DE LA QUE SE RECIBEN APOYOS.
                                                                                        desc_ins
inst_enc                                                                                        
1         Institución encargada de coordinar y/o ejecutar el programa de modernización catastral
2                                                        institución de la que se reciben apoyos


In [392]:
P0306_01t1 = P0306_01t[P0306_01t['inst_enc'] == 1]['tip_inst'].groupby('ubic_geo').apply(list).rename('i_coord_ejecuta')
P0306_01t2 = P0306_01t[P0306_01t['inst_enc'] == 2]['tip_inst'].groupby('ubic_geo').apply(list).rename('i_otorga_apoyos')

In [393]:
P0306_01t1.head()

ubic_geo
1006                                   [OTRA INSTITUCION]
2001                                             [SEDATU]
2002                                             [SEDATU]
2003    [Administración pública de la entidad federativa]
2004                                   [OTRA INSTITUCION]
Name: i_coord_ejecuta, dtype: object

In [326]:
P0306_01t2.head()

ubic_geo
1006    [OTRA INSTITUCION]
2001              [SEDATU]
2002              [SEDATU]
2003              [SEDATU]
2004    [OTRA INSTITUCION]
Name: i_otorga_apoyos, dtype: object

Finalmente, se unirán todas las series en un solo dataframe

In [394]:
# Convertir series en Dataframes
P0306_00 = P0306_00.to_frame()
P0306_03 = P0306_03.to_frame()
P0306_02 = P0306_02.to_frame()
P0306_02b = P0306_02b.to_frame()
P0306_01t1 = P0306_01t1.to_frame()
P0306_01t2 = P0306_01t2.to_frame()

In [407]:
# Unir dataframes
P0306 = P0306_00.join(P0306_03).join(P0306_02).join(P0306_02b).join(P0306_01t1).join(P0306_01t2)
P0306 = P0306.where((pd.notnull(P0306)), None)

In [408]:
P0306.head()

,prog_mod,perio_ac,acc_modr,n_acc_modr,i_coord_ejecuta,i_otorga_apoyos
ubic_geo,,,,,,
15045,1,2,"[1, 2, 4, 5, 6]",5,[Administración pública de la entidad federati...,[BANOBRAS]
15046,2,None,None,None,None,None
15047,2,None,None,None,None,None
15048,1,2,"[1, 2, 3, 4, 6]",5,[SEDATU],[Administración pública de la entidad federativa]
15049,2,None,None,None,None,None


__Metadatos para P0306__

In [ ]:
P0306meta = 